**Tuning Takens + Transfer Entropy (Keras-Tuner)**
This notebook performs hyperparameter search for the TakensConv1D block (dx, dy, tau, mu), which is used to build Takens embeddings and compute Transfer Entropy (TE) between EEG channels

# Setup (dependencies and data)

In [1]:
!pip show tensorflow tensorflow-probability
!pip install tensorflow-probability==0.24.0
!pip install --upgrade keras-tuner
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git
!pip install mne #The MNE Package is installed
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip -y #Package with useful functions for motor imagery classification based in EEG.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!dir

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras
---
Name: tensorflow-probability
Version: 0.25.0
Summary: Probabilistic modeling and statistical inference in TensorFlow
Home-page: http://github.com/tensorflow/probability
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, cloudpickle, decorator, dm-tree, gast, numpy, six
Required-by: dopamine_r

# Imports and utilities

In [19]:
import numpy as np
import os
import itertools
import random
import pickle
import mne
import h5py
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import keras_tuner as kt


from gcpds.databases.BCI_Competition_IV import Dataset_2a
from typing import Sequence, Tuple
from scipy.signal import iirnotch, filtfilt, butter, freqz
import matplotlib.pyplot as plt
from scipy.spatial import distance
import networkx as nx
from tqdm import tqdm
from mne.preprocessing import compute_current_source_density
from mne.channels import make_standard_montage, read_custom_montage
from scipy.signal import butter, filtfilt, resample, iirnotch
from gcpds.visualizations.series import plot_eeg
from scipy.stats import norm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from tqdm import tqdm

from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from gcpds.databases import GIGA_MI_ME
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import LeavePGroupsOut, StratifiedGroupKFold
from tensorflow.keras.models import Model
from scipy.spatial.distance import cdist
from sklearn.model_selection import GroupKFold
from tensorflow.keras.models import load_model

from keras_tuner import Objective
from keras_tuner import HyperModel
from keras.layers import Layer, Activation
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.metrics import BinaryAccuracy, Recall

In [20]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Load EEG data for a specific subject with preprocessing, including filtering and surface Laplacian.
    The signal is cropped to the 2–4 second interval, and only the left-hand (0) and right-hand (1) classes are included
    
    Args:
        db (Dataset_2a): Dataset object.
        sbj (int): Subject identifier (1–9).
        mode (str): 'training' or 'testing'.
        fs (float): Sampling frequency.

    Returns:
        tuple: Preprocessed EEG data with surface Laplacian and time-cropped segment (X), and corresponding labels (y).
    """
    # Load the subject's data
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()# Data and labels
    X = X[:, :-3, :]# Select only the EEG channels (22 channels)
    X = X * 1e6# Convert to microvolts

    # Apply a notch filter (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Apply a band-pass filter (0.5–100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Crop the data between 2 and 4 seconds
    start_sample = int(2 * fs)# Start at 2 seconds
    end_sample = int(4 * fs)# End at 4 seconds
    X = X[:, :, start_sample:end_sample]

    # Keep only the classes of interest (1: left hand, 2: right hand)
    clases = [0, 1]
    mask = np.isin(y, clases)
    X = X[mask]
    y = y[mask]

    # List of the 22 EEG channel names (excluding EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Create channel information for the EEG channels
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  
        ch_types=["eeg"] * len(eeg_channel_names) 
    )

    # Load a standard montage based on the 10–20 system
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Apply surface Laplacian computation 
    laplacian_X = []
    for trial in X:
        # Create a RawArray object for each trial
        raw = mne.io.RawArray(trial, info)
        # Compute the surface Laplacian
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Get the data with the Laplacian applied
        laplacian_X.append(raw.get_data())

    X = np.stack(laplacian_X)

    return X, y

# Custom layers and metrics

In [24]:
@register_keras_serializable(package="CustomLayers")
class TakensConv1D(tf.keras.layers.Layer):
    def __init__(self, dx=5, dy=5, tau=1, mu=4, **kwargs):
        super().__init__(**kwargs)
        self.dx = int(dx)
        self.dy = int(dy)
        self.tau = int(tau)
        self.mu = int(mu)
        self.num_filters = self.dx + self.dy + 1

    def build(self, input_shape):
        kernel_size = self.mu + (self.dx - 1) * self.tau + 1
        kernel_shape = (kernel_size, 1, self.num_filters)
        kernel = tf.zeros(kernel_shape, dtype=tf.float32)

        offsets_x = self.mu + tf.range(self.dx) * self.tau
        offsets_y = tf.range(1, self.dy + 1) * self.tau
        offset_y_t = tf.constant([0], dtype=tf.int32)

        filas_x = tf.range(self.dx)
        filas_y = self.dx + tf.range(self.dy)
        fila_y_t = tf.constant([self.dx + self.dy])

        cols_x = filas_x
        cols_y = filas_y
        col_y_t = fila_y_t

        idx_x = tf.stack([filas_x, offsets_x, cols_x], axis=1)
        idx_y = tf.stack([filas_y, offsets_y, cols_y], axis=1)
        idx_yt = tf.stack([fila_y_t, offset_y_t, col_y_t], axis=1)

        indices_total = tf.concat([idx_x, idx_y, idx_yt], axis=0)
        updates = tf.ones([tf.shape(indices_total)[0]], dtype=tf.float32)

        sort_order = tf.argsort(indices_total[:, 0])
        indices_sorted = tf.gather(indices_total, sort_order)

        row_for_scatter = tf.cast(indices_sorted[:, 1], tf.int32)
        col_for_scatter = tf.cast(indices_sorted[:, 2], tf.int32)

        final_indices = tf.stack([row_for_scatter,
                                  tf.zeros_like(row_for_scatter),
                                  col_for_scatter], axis=1)

        kernel = tf.scatter_nd(final_indices, updates, kernel_shape)
        self.kernel = kernel.numpy()[::-1]

        self.conv1d = tf.keras.layers.Conv1D(
            filters=self.num_filters,
            kernel_size=kernel_size,
            strides=self.tau,
            padding="valid",
            use_bias=False
        )
        self.conv1d.build((None, input_shape[2], 1))
        self.conv1d.set_weights([self.kernel])
        self.conv1d.trainable=False

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        channels = tf.shape(inputs)[1]
        time_steps = tf.shape(inputs)[2]

        reshaped = tf.reshape(inputs, (-1, time_steps, 1))
        conv_output = self.conv1d(reshaped)
        new_time = tf.shape(conv_output)[1]

        output = tf.reshape(conv_output, 
                            (batch_size, channels, new_time, self.num_filters))

        x_sub_t_minus_mu = output[..., :self.dx]
        y_sub_t_minus_1 = output[..., self.dx:self.dx + self.dy]
        y_sub_t = output[..., -1:]

        return x_sub_t_minus_mu, y_sub_t_minus_1, y_sub_t
       

@register_keras_serializable(package="CustomLayers")
class KernelLayer(tf.keras.layers.Layer):
    def __init__(self, 
                 amplitude=1.0,
                 trainable_amplitude=False, 
                 length_scale=1.0,
                 trainable_length_scale=False,
                 alpha=1.0,  # Only used for Rational Quadratic
                 trainable_alpha=False,
                 kernel_type="gaussian",  # "gaussian" or "rational_quadratic"
                 **kwargs):
        super(KernelLayer, self).__init__(**kwargs)

        self.init_amplitude = amplitude
        self.trainable_amplitude = trainable_amplitude
        self.init_length_scale = length_scale
        self.trainable_length_scale = trainable_length_scale
        self.init_alpha = alpha
        self.trainable_alpha = trainable_alpha
        self.kernel_type = kernel_type.lower()

    def build(self, input_shape):
        self.amplitude = self.add_weight(
            name="amplitude",
            shape=(),
            initializer=tf.constant_initializer(self.init_amplitude),
            trainable=self.trainable_amplitude,
            dtype=self.dtype
        )

        self.length_scale = self.add_weight(
            name="length_scale",
            shape=(),
            initializer=tf.constant_initializer(self.init_length_scale),
            trainable=self.trainable_length_scale,
            dtype=self.dtype
        )

        if self.kernel_type == "rational_quadratic":
            self.alpha = self.add_weight(
                name="alpha",
                shape=(),
                initializer=tf.constant_initializer(self.init_alpha),
                trainable=self.trainable_alpha,
                dtype=self.dtype
            )

        super(KernelLayer, self).build(input_shape)

    def call(self, X):
        if self.kernel_type == "gaussian":
            kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale
            )
        elif self.kernel_type == "rational_quadratic":
            kernel = tfp.math.psd_kernels.RationalQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale,
                scale_mixture_rate=self.alpha
            )
        else:
            raise ValueError(f"Unsupported kernel_type: {self.kernel_type}")
        
        return kernel.matrix(X, X)
       
@register_keras_serializable(package="CustomLayers")
class TransferEntropyLayer(tf.keras.layers.Layer):
    def __init__(self, alpha=2, **kwargs):

        super().__init__(**kwargs)
        self.alpha = int(alpha)

    def compute_entropy(self, K_hadamard):
        
        trace_hadamard = tf.reduce_sum(tf.linalg.diag_part(K_hadamard), axis=-1) 
        trace_hadamard = tf.expand_dims(tf.expand_dims(trace_hadamard, axis=-1), axis=-1)
        K_normalized = K_hadamard / trace_hadamard

        K_power = tf.linalg.matmul( K_normalized , K_normalized, grad_a=True, grad_b=True ) 
        trace_power = tf.reduce_sum(tf.linalg.diag_part(K_power), axis=-1)  
        H_alpha = (1 / (1 - self.alpha)) * tf.math.log(trace_power)
        return H_alpha

    def call(self, K_x, K_y_minus_1, K_y):
        
        K_x_exp = tf.expand_dims(K_x, axis=2)
        
        K_y_minus_1_exp = tf.expand_dims(K_y_minus_1, axis=1)
        K_y_exp = tf.expand_dims(K_y, axis=1)

        
        H_1 = self.compute_entropy(K_y_minus_1_exp * K_x_exp)
        H_2 = self.compute_entropy(K_y_exp * K_y_minus_1_exp * K_x_exp)
        H_3 = self.compute_entropy(K_y_exp * K_y_minus_1_exp)
        H_4 = self.compute_entropy(K_y_minus_1_exp)

        TE = H_1 - H_2 + H_3 - H_4
        
        return TE
        
@register_keras_serializable(package="CustomLayers")      
class RemoveDiagonalFlatten(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
    
        super().__init__(**kwargs)

    def call(self, inputs):
        
        shape_dyn = tf.shape(inputs)  
        batch_size = shape_dyn[0]
        c = tf.shape(inputs)[1]

        
        tf.debugging.assert_equal(
            tf.shape(inputs)[1], tf.shape(inputs)[2],
            message="RemoveDiagonalFlatten: la matriz de entrada no es cuadrada."
        )  
        diag_mask = tf.eye(c, dtype=inputs.dtype)  
        inputs_no_diag = inputs * (1 - diag_mask) 
        flattened = tf.reshape(inputs_no_diag, [batch_size, -1])  
        non_diag = tf.boolean_mask(flattened, tf.reshape(1 - diag_mask, [-1]), axis=1)
        num_features = c * (c - 1)  
        result = tf.reshape(non_diag, [batch_size, num_features])  

        return result

# Additional custom metrics (Sensitivity/Specificity, etc.)

In [26]:
from tensorflow.keras.metrics import Metric
@register_keras_serializable(package="CustomMetrics")
class Sensitivity(Metric):
    def __init__(self, name='sensitivity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class Accuracy(Metric):
    def __init__(self, name='accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        num = self.tp.result() + self.tn.result()
        den = num + self.fp.result() + self.fn.result() + tf.keras.backend.epsilon()
        return num / den

    def reset_states(self):
        self.tp.reset_states()
        self.tn.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.tp.result() / (self.tp.result() + self.fp.result() + tf.keras.backend.epsilon())
        recall = self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class Specificity(Metric):
    def __init__(self, name='specificity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.tn.result() / (self.tn.result() + self.fp.result() + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tn.reset_states()
        self.fp.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [22]:
# Create a dataset instance
db = Dataset_2a("/kaggle/input/dataset-2a")
fs = 250.0

# Load the subject's data in 'training' mode
X, y = load_BCICIV2a(db, sbj=5, mode="training", fs=fs)
print("X shape:", X.shape)
print("y shape:", y.shape)

tamaño de X: (129, 22, 500)
tamaño de y: (129,)


In [ ]:
train_data, val_data, train_labels, val_labels = train_test_split(
    X, y,
    test_size=0.2,
    random_state=72,
    stratify=y
)

def create_tunable_model(dx, dy, tau, mu,
                         kernel_type, learning_rate,
                         kernel_size):
    input_eeg = tf.keras.Input(shape=(22, 500), name="input_eeg")

    # Block 1 with tunable kernel_size
    x = tf.keras.layers.DepthwiseConv1D(
        kernel_size=kernel_size,
        strides=1,
        padding="valid",
        activation="relu",
        depth_multiplier=1,
        data_format="channels_first",
        name="block1_depthwise_conv1d"
    )(input_eeg)

    x = tf.keras.layers.AveragePooling1D(
        pool_size=4,
        strides=4,
        padding="valid",
        data_format="channels_first",
        name="block1_avg_pooling"
    )(x)

    x = tf.keras.layers.BatchNormalization(
        axis=1,
        name="block1_batch_norm"
    )(x)

    # Block 2: TakensConv1D
    takens = TakensConv1D(dx=dx, dy=dy, tau=tau, mu=mu,
                          name="takens_conv1d")(x)
    x_sub, y_minus_1, y_t = takens

    # Dense projections
    x_sub     = tf.keras.layers.Dense(dx, activation=None,
                                      use_bias=False,
                                      name="dense_proj_x")(x_sub)
    y_minus_1 = tf.keras.layers.Dense(dy, activation=None,
                                      use_bias=False,
                                      name="dense_proj_y_1")(y_minus_1)
    y_t       = tf.keras.layers.Dense(1, activation=None,
                                      use_bias=False,
                                      name="dense_proj_y")(y_t)

    # Fixed kernel layers
    def fixed_kernel(name):
        layer = KernelLayer(
            amplitude=1.0, trainable_amplitude=False,
            length_scale=1.0, trainable_length_scale=False,
            alpha=1.0, trainable_alpha=False,
            kernel_type=kernel_type,
            name=name
        )
        layer.trainable = False
        return layer

    Kx  = fixed_kernel("kernel_x")(x_sub)
    Ky1 = fixed_kernel("kernel_y_minus_1")(y_minus_1)
    Ky  = fixed_kernel("kernel_y")(y_t)

    # Transfer Entropy + flattening
    TE   = TransferEntropyLayer(alpha=2,
                                name="transfer_entropy")(Kx, Ky1, Ky)
    flat = RemoveDiagonalFlatten(name="remove_diag_flatten")(TE)

    # Output head
    h   = tf.keras.layers.Dense(10, activation="relu",
                                name="dense_1")(flat)
    out = tf.keras.layers.Dense(1, activation="sigmoid",
                                name="output")(h)

    model = tf.keras.Model(inputs=input_eeg, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=[
            BinaryAccuracy(name="accuracy"),
            F1Score(name="f1_score"),
            Recall(name="sensitivity"),
            Specificity(name="specificity")
        ]
    )
    return model

def build_model(hp):
    # Hyperparameters to search
    dx          = hp.Int("dx",  min_value=1,  max_value=10, step=1)
    dy          = hp.Int("dy",  min_value=1,  max_value=10, step=1)
    tau         = hp.Int("tau", min_value=1,  max_value=5,  step=1)
    mu          = hp.Int("mu",  min_value=0,  max_value=10, step=1)
    lr          = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
    kernel_type = "rational_quadratic"

    # New: searchable range for kernel_size
    kernel_size = hp.Int(
        "kernel_size",
        min_value=3,      # very small windows (3 samples)
        max_value=125,    # up to ~125 samples (~0.25 s at 500 Hz)
        step=2            # odd values only
    )

    # Validate that the Takens embedding window fits after Conv+Pool
    conv_len = 500 - kernel_size + 1
    pool_len = (conv_len - 4) // 4 + 1
    window   = mu + (dx - 1) * tau + 1
    if window > pool_len:
        # Invalid trial: return a trivial model to penalize it
        m = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1, activation="sigmoid")
        ])
        m.compile("adam", "binary_crossentropy", ["accuracy"])
        return m

    # Build and return the actual model
    return create_tunable_model(
        dx, dy, tau, mu,
        kernel_type, lr,
        kernel_size
    )

# Tuner configuration
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective=kt.Objective("val_f1_score", direction="max"),
    max_trials=50,
    executions_per_trial=2,
    directory="tuner_dir5",
    project_name="takens_te_tuning_5"
)

stop_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=1
)

# Start the search
tuner.search(
    x=train_data,
    y=train_labels,
    validation_data=(val_data, val_labels),
    epochs=200,
    callbacks=[stop_early]
)

# Extract the best result
best_hp    = tuner.get_best_hyperparameters(num_trials=1)[0]
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_score = best_trial.score  # val_f1_score

# Save to a DataFrame and CSV
import pandas as pd

best_dict = {
    "dx":            best_hp.get("dx"),
    "dy":            best_hp.get("dy"),
    "tau":           best_hp.get("tau"),
    "mu":            best_hp.get("mu"),
    "kernel_size":   best_hp.get("kernel_size"),
    "learning_rate": best_hp.get("learning_rate"),
    "val_f1_score":  best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)
df_best.to_csv("best_hyperparameters_with_score5.csv", index=False)

Trial 32 Complete [00h 01m 01s]
val_f1_score: 0.6376811265945435

Best val_f1_score So Far: 0.7126436233520508
Total elapsed time: 00h 33m 23s

Search: Running Trial #33

Value             |Best Value So Far |Hyperparameter
1                 |1                 |dx
7                 |8                 |dy
5                 |5                 |tau
3                 |3                 |mu
0.01              |0.01              |learning_rate
103               |65                |kernel_size

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.4640 - f1_score: 0.5111 - loss: 0.6928 - sensitivity: 0.5463 - specificity: 0.3788 - val_accuracy: 0.5000 - val_f1_score: 0.1333 - val_loss: 0.7096 - val_sensitivity: 0.0769 - val_specificity: 0.9231
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7662 - f1_score: 0.7511 - loss: 0.6653 - sensitivity: 0.6598 - specificity: 0.8961 - val_accuracy: 0.5769 - val_f1_score: 0.5217 - val_loss: 0.6670 - val_sensitivity: 0.4615 - val_s

In [ ]:
# 1) Retrieve the best HyperParameters and Trial
best_hp    = tuner.get_best_hyperparameters(num_trials=1)[0]
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

# 2) Extract the validation score (F1)
best_score = best_trial.score  # corresponds to val_f1_score

# 3) Build a DataFrame with all values
import pandas as pd

best_dict = {
    "dx":            best_hp.get("dx"),
    "dy":            best_hp.get("dy"),
    "tau":           best_hp.get("tau"),
    "mu":            best_hp.get("mu"),
    "learning_rate": best_hp.get("learning_rate"),
    "val_f1_score":  best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)

# 4) Save to CSV
df_best.to_csv("best_hyperparameters_with_score5.csv", index=False)

In [12]:
# 1) Iterate over all trials stored in the tuner
records = []
for trial in tuner.oracle.trials.values():
    # Each trial has a HyperParameters object where .values is a dict of {hp_name: value}
    hp_dict = trial.hyperparameters.values.copy()
    # Add this trial's score (val_f1_score)
    hp_dict["val_f1_score"] = trial.score
    # Optional: identify the trial
    hp_dict["trial_id"] = trial.trial_id
    records.append(hp_dict)

# 2) Create the DataFrame
df_all = pd.DataFrame(records)

# 3) Sort by the metric in descending order to show the best trials first
df_all = df_all.sort_values("val_f1_score", ascending=False).reset_index(drop=True)

# 4) Display and save
print(df_all)
df_all.to_csv("all_trials_hyperparameters5.csv", index=False)

    dx  dy  tau  mu  learning_rate  val_f1_score trial_id
0    5   1    4   3         0.0001      0.905983       49
1    5   1    4   3         0.0001      0.902778       43
2    5   1    4   2         0.0001      0.893793       37
3    5   1    4   3         0.0001      0.873016       47
4    5   2    4   3         0.0001      0.851429       28
5    5   1    4   3         0.0001      0.841111       44
6    9   2    2   8         0.0001      0.831351       12
7    1   9    3   6         0.0001      0.830629       19
8   10   9    4   5         0.0010      0.828722       01
9    1   9    3   4         0.0001      0.822511       26
10   1   9    3   5         0.0001      0.815686       04
11   5   2    4   4         0.0001      0.812890       08
12   1   9    3   5         0.0001      0.811111       31
13   5   1    4   3         0.0001      0.795796       39
14  10   9    4   5         0.0010      0.786134       33
15   5   1    3   9         0.0100      0.782407       10
16   5   1    

In [17]:
# 1) Get the already-trained model with the best hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]

# 2) Save it to /kaggle/working
best_model.save("best_hp_model5.keras")


# Create a dataset instance
db = Dataset_2a("/kaggle/input/dataset-2a")
fs = 250.0

# Load the subject's data in 'evaluation' mode
X, y = load_BCICIV2a(db, sbj=5, mode="evaluation", fs=fs)
print("X shape:", X.shape)
print("y shape:", y.shape)

results = best_model.evaluate(X, y, return_dict=True)

print("Evaluation results:")
print(f"  Accuracy:     {results['accuracy']:.4f}")
print(f"  F1 Score:     {results['f1_score']:.4f}")
print(f"  Loss:         {results['loss']:.4f}")
print(f"  Sensitivity:  {results['sensitivity']:.4f}")
print(f"  Specificity:  {results['specificity']:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


tamaño de X: (130, 22, 500)
tamaño de X: (130,)
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 420ms/step - accuracy: 0.7763 - f1_score: 0.7799 - loss: 0.5086 - sensitivity: 0.8985 - specificity: 0.6774
Resultados evaluación:
  Accuracy:     0.7846
  F1 Score:     0.8056
  Loss:         0.5000
  Sensitivity:  0.8923
  Specificity:  0.6769


# Training with Quadratic

In [18]:
model = load_model("/kaggle/working/best_hp_model5.keras", compile=True)

# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=100,
    restore_best_weights=True,
    verbose=1
)
csv_logger = CSVLogger("continued_training.log", append=True)

# 3) Load the BCI Competition IV-2a data
db = Dataset_2a("/kaggle/input/dataset-2a")
fs = 250.0

#    – Training split
X_full, y_full = load_BCICIV2a(db, sbj=5, mode="training", fs=fs)
#    – Evaluation split (final test)
X_test, y_test = load_BCICIV2a(db, sbj=5, mode="evaluation", fs=fs)

# 4) Split the training set into train/val (80/20)
train_data, val_data, train_labels, val_labels = train_test_split(
    X_full, y_full,
    test_size=0.2,
    random_state=42,
    stratify=y_full
)

print(f"Train: {train_data.shape}, {train_labels.shape}")
print(f"Val:   {val_data.shape}, {val_labels.shape}")
print(f"Test:  {X_test.shape}, {y_test.shape}")

# 5) Continue training
history = model.fit(
    x=train_data,
    y=train_labels,
    validation_data=(val_data, val_labels),
    epochs=1000,
    callbacks=[early_stopping, csv_logger]
)

# 6) Evaluate on the evaluation set
results = model.evaluate(X_test, y_test, return_dict=True)
print("Test results:")
for name, value in results.items():
    print(f"  {name}: {value:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Train: (92, 22, 500), (92,)
Val:   (24, 22, 500), (24,)
Test:  (130, 22, 500), (130,)
Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 0.9090 - f1_score: 0.9208 - loss: 0.4373 - sensitivity: 0.9724 - specificity: 0.8333 - val_accuracy: 0.8750 - val_f1_score: 0.8889 - val_loss: 0.4484 - val_sensitivity: 0.9231 - val_specificity: 0.8182
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.8777 - f1_score: 0.8985 - loss: 0.4493 - sensitivity: 0.9604 - specificity: 0.7700 - val_accuracy: 0.8750 - val_f1_score: 0.8889 - val_loss: 0.4477 - val_sensitivity: 0.9231 - val_specificity: 0.8182
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8808 - f1_score: 0.8963 - loss: 0.4499 - sensitivity: 0.9510 - specificity: 0.7983 - val_accuracy: 0.8750 - val_f1_score: 0.8889 - val_loss: 0.4472 - val_sensitivity: 0.9231 - val_specificity: 0.8182
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9042 - f1_score: 0.9154 - loss: 0.4432 - sensitivity: 0